In [ ]:
%pip install -U py3dti miniaudio soundcard

In [ ]:
from math import sin, cos, radians, ceil
from IPython.display import Audio
import numpy as np
from miniaudio import decode_file, SampleFormat
import soundcard
import py3dti

## Setup py3dti

In [ ]:
renderer = py3dti.BinauralRenderer(rate=44100, buffer_size=512, resampled_angular_resolution=5)

### Add listener

In [ ]:
listener = renderer.add_listener(position=None, orientation=None, head_radius=0.0875)
listener.load_hrtf_from_sofa(f'../3dti_AudioToolkit/resources/HRTF/SOFA/3DTI_HRTF_IRC1008_512s_{renderer.rate}Hz.sofa')
# listener.load_hrtf_from_3dti(f'../3dti_AudioToolkit/resources/HRTF/3DTI/3DTI_HRTF_IRC1008_512s_{renderer.rate}Hz.3dti-hrtf') # faster, but less common file format
listener.load_ild_near_field_effect_table(f'../3dti_AudioToolkit/resources/ILD/NearFieldCompensation_ILD_{renderer.rate}.3dti-ild')

In [ ]:
listener.position, listener.orientation

In [ ]:
listener.position = (0, 0, 0) # default listener position
listener.orientation = (1, 0, 0, 0) # default listener orientation

### Add environment

In [ ]:
environment = renderer.add_environment()
environment.load_brir_from_sofa(f'../3dti_AudioToolkit/resources/BRIR/SOFA/3DTI_BRIR_medium_{renderer.rate}Hz.sofa')
# environment.load_brir_from_3dti(f'../3dti_AudioToolkit/resources/BRIR/3DTI/3DTI_BRIR_medium_{renderer.rate}Hz.3dti-brir') # faster, but less common file format

### Add source(s)

In [ ]:
source = renderer.add_source(position=(2, 2, 2))

In [ ]:
source.position

### Display variables

In [ ]:
renderer, renderer.listener, renderer.sources, renderer.environments

## Read source samples

In [ ]:
file_path = f'../3dti_AudioToolkit/resources/AudioSamples/Anechoic Speech {renderer.rate}.wav'
decoded_file = decode_file(filename=file_path, output_format=SampleFormat.FLOAT32,
                           nchannels=1, sample_rate=renderer.rate)
samples = np.asarray(decoded_file.samples)

## Offline Rendering to `np.array`

### Static sources and listener

In [ ]:
sources = {source: samples}
binaural_samples = renderer.render_offline(sources)

In [ ]:
Audio(binaural_samples.T, rate=renderer.rate, normalize=False)

### Dynamic sources or listener

In [ ]:
# let the source circle counter-clockwise in the frontal plane
source_positions = {source: [(0, sin(radians(i)), cos(radians(i))) for i in range(ceil(len(samples)/renderer.buffer_size))]}
binaural_samples = renderer.render_offline(sources, source_positions)

In [ ]:
Audio(binaural_samples.T, rate=renderer.rate, normalize=False)

## Block-Wise Streaming Output

### Source samples available offline

#### Static sources and listener

In [ ]:
binaural_streamer = renderer.render_online(sources)
type(binaural_streamer)

In [ ]:
with soundcard.default_speaker().player(samplerate=renderer.rate, channels=2) as stereo_speaker:
    for block_idx in range(len(binaural_streamer)):
        stereo_speaker.play(binaural_streamer())

In [ ]:
binaural_streamer()

#### Dynamic sources or listener

In [ ]:
binaural_streamer = renderer.render_online(sources)
with soundcard.default_speaker().player(samplerate=renderer.rate, channels=2) as stereo_speaker:
    for block_idx in range(len(binaural_streamer)):
        # let source circle counter-clockwise in the horizontal plane
        position_map = {source: (cos(radians(block_idx)), sin(radians(block_idx)), 0)}
        stereo_speaker.play(binaural_streamer(position_map))

In [ ]:
binaural_streamer = renderer.render_online(sources)
binaural_blocks = []
for block_idx in range(len(binaural_streamer)):
    # let source circle clockwise in the median plane
    position_map = {source: (cos(radians(block_idx)), 0, sin(radians(block_idx)))}
    binaural_blocks.append(binaural_streamer(position_map))
binaural_samples = np.row_stack(binaural_blocks)

In [ ]:
Audio(binaural_samples.T, rate=renderer.rate, normalize=False)

### Source samples incoming block by block

#### Dynamic sources or listener

In [ ]:
binaural_streamer = renderer.render_online()
type(binaural_streamer)

In [ ]:
with soundcard.default_speaker().player(samplerate=renderer.rate, channels=2) as stereo_speaker:
    sample_idx = 0
    while True:
        input_buffer = samples[sample_idx:sample_idx+renderer.buffer_size]
        # fill remainder of input buffer with zeros
        input_buffer = np.concatenate((input_buffer, np.zeros(renderer.buffer_size-len(input_buffer))))
        samples_map = {source: input_buffer}
        position_map = {source: (cos(radians(sample_idx/renderer.buffer_size)), sin(radians(sample_idx/renderer.buffer_size)), 0)}
        output_buffer = binaural_streamer(samples_map, position_map)
        stereo_speaker.play(output_buffer)
        sample_idx = (sample_idx + renderer.buffer_size) % len(samples)

## Some more (modifiable) properties with sensible default values

In [ ]:
listener.head_radius, listener.ild_attenuation

In [ ]:
source.spatialization_mode, source.anechoic_processing, source.reverb_processing

In [ ]:
source.far_distance_effect, source.near_field_effect

In [ ]:
source.propagation_delay, source.anechoic_distance_attenuation, source.anechoic_distance_attenuation_smoothing